In [3]:
import time
import numpy as np
import pandas as pd
from perato_utils import is_pareto
from gpr import process_generation_2DEI
from ei import getEiVec2D_aug, getPiVec2D_aug
from plot_utils import *

In [4]:
import sys
import sklearn.preprocessing
from scipy import stats
import pandas as pd
import colorlover as cl
import numpy as np
from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
import plotly.io as pio
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.io.orca.config.executable = '/anaconda2/envs/pytorch/bin/orca'
init_notebook_mode(connected=True)

glob_layout = go.Layout(
    font=dict(family='Helvetica', size=24, color='black'),
    margin=dict(l=100, r=10, t=10, b=100),
    xaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    yaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    legend_orientation="v",
)

In [5]:
def fun1(x):
    return (1/((1-0.15)**2))*((x-0.10)**2 )

def fun2(x):
#     return (1/((0.75)**2))*((x-0.75)**2 )
    return np.exp((-0.3*np.sin(3/2*(np.pi*x/5))**2 + x))/26.6222

In [6]:
def fun1(x):
    return (1/((1-0.15)**2))*((x-0.10)**2 )

def fun2(x):
    return (15*np.sin(3/2*(np.pi*x/5))**2 + x)/26.6222

In [7]:
x_det = np.linspace(0, 1, 50)
x_det_mean, x_det_std = np.mean(x_det), np.std(x_det)
x_det = (x_det- x_det_mean)/x_det_std

In [8]:
y1 = fun1(x_det*x_det_std + x_det_mean)
y2 = fun2(x_det*x_det_std + x_det_mean)

In [9]:
res = {"x": x_det, "y1": y1, "y2": y2}
df = pd.DataFrame(data=res)

In [10]:
pareto_inds, pareto_points = is_pareto(points=df[['y1', 'y2']].values)

In [11]:
plot_pareto_front(df, pareto_points, figname='2DEI_example/global_pareto_front.pdf')

In [12]:
start_n = 2
initial_points = df.sample(n=start_n, random_state=0)
true_points = initial_points
## loop control
known_points = initial_points
true_eval = True
i = 1
tb_thres, tb_count = 0, 0

In [ ]:
while tb_count < 10:
    pareto_inds_known, pareto_points_known = is_pareto(points=known_points[['y1', 'y2']].values)
    df, _, _ = process_generation_2DEI(df_known=known_points, df=df, fnames=['x'])
    df['known'] = [True if idx in list(known_points.index) else False for idx, row in df.iterrows()]
    plot_model(df, known_points, 
              figname='2DEI_example/model_pred_gen-%d.pdf'%tb_count,
              show=False)
    time.sleep(1)
    df = getEiVec2D_aug(df, approx_pareto_list=pareto_points_known)
    new_x = df.iloc[df['pi'].idxmax( axis=0, skipna=True,)]
    plot_known_and_new(df, known_points, new_x,
                      figname='2DEI_example/points_explored_gen-%d.pdf'%tb_count,
                      show=False)
    time.sleep(1)
    tb_count += 1
    known_points = known_points.append(new_x[known_points.columns])